In [1]:
import re
import string
from unidecode import unidecode
import pandas as pd

In [2]:
invalid_char = set(string.punctuation)
# invalid_char

In [3]:
f = open('VDic_uni.txt', encoding="utf8")
wordlist = []
while True:
    line = f.readline()
    if not line:
        break
    
    # print(line)
    # print(line.strip())
    tmp = line.split("\t")[0].replace("\n", "") 
    tmp = tmp.replace("-", " ")
    w = tmp.split(' ')
    # print(w)
    # w = [s.lower().replace("\ufeff", "") for s in w if (len(s) != 0  and s[0] not in invalid_char) ]
    # for s in w:
    #     if len(s) != 0 and s[0] not in invalid_char and all(char != ',' for char in s):
    #         s.lower().replace("\ufeff", "")
    #     else:
    #         print(s)


    w = [s.lower().replace("\ufeff", "") for s in w if len(s) != 0 and s[0] not in invalid_char and all(char != ',' for char in s)]

    wordlist += w

SyllableLs = set(wordlist)
# print('đất,' in SyllableLs)
# print(SyllableLs)
print("Number of syllable in VDic: ", len(SyllableLs))

Number of syllable in VDic:  7996


In [4]:
phu_am_dau = ['th', 'p', 't', 'tr', 'ch', 'c', 'k', 'q', 'b', 'đ', 'ph', 'x', 's', 'kh', 'h', 'v', 'd', 'gi', 'r', 'g', 'gh', 'm', 'n', 'nh', 'ng', 'ngh', 'l']
print("PAD: ", len(phu_am_dau))
am_dem = ['u', 'o']
print("Am Dem: ", len(am_dem))
am_chinh = ['i', 'y', 'ư', 'u', 'iê', 'ia', 'yê', 'ya', 'ươ', 'ưa', 'uê', 'ua', 'ê', 'ơ', 'â', 'ô', 'e','a', 'ă', 'o' , 'uô', 'oo']
print("Am Chinh: ", len(am_chinh))
am_cuoi = ['m', 'n', 'ng', 'nh', 'p', 't', 'ch', 'c', 'u', 'o', 'i', 'y']
print("Am Cuoi: ", len(am_cuoi))
thanh = 6
numOfSyll = len(phu_am_dau)*len(am_chinh)*len(am_dem)*len(am_cuoi)*thanh
print("So am tiet co the co: ", numOfSyll) 

PAD:  27
Am Dem:  2
Am Chinh:  22
Am Cuoi:  12
So am tiet co the co:  85536


=>  Có sự chênh lệch giữa số âm tiết trong từ điển (7996) và số âm tiết ước tính theo tổ hợp [PÂĐ x Đệm x Chính x Cuối x Thanh] (85536) là vì:
<br> + Có những tổ hợp không có nghĩa nên không xuất hiện trong từ điển
<br>

In [6]:
def no_accent_vnm(s):
    s = re.sub(r'[àáạảã]', 'a', s)
    s = re.sub(r'[ầấậẩẫ]', 'â', s)
    s = re.sub(r'[ằắặẳẵ]', 'ă', s)
    s = re.sub(r'[èéẹẻẽ]', 'e', s)
    s = re.sub(r'[ềếệểễ]', 'ê', s)
    s = re.sub(r'[òóọỏõ]', 'o', s)
    s = re.sub(r'[ồốộổỗ]', 'ô', s)
    s = re.sub(r'[ờớợởỡ]', 'ơ', s)  
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ùúụủũ]', 'u', s)
    s = re.sub(r'[ừứựửữ]', 'ư', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    return s

In [35]:
def separatePhoneme(word):
    word = no_accent_vnm(word)
    print(word)
    # Lấy phụ âm đầu: có thể 2 ký tự hoặc 1
    cnt = 0 
    k = -1
    phuAmDau, amDem, amChinh, amCuoi = -1, -1, -1, -1
    if (len(word) > 1):
        tmp = word[-2] + word[-1]
        if tmp in am_cuoi:
            amCuoi = tmp # ng
            cnt = 2
            k = -3
        elif (tmp not in am_cuoi and word[-1] in am_cuoi):
            amCuoi = word[-1]
            cnt = 1
            k = -2
    print (cnt)
    print(k)  
    # Lấy âm chính: 
    if len(word) - cnt > 1:
        tmp = word[k-1] + word[k]
        if tmp in am_chinh:
            amChinh = tmp
            k = k - 2
            cnt += 2
        elif tmp not in am_chinh and word[k] in am_chinh:
            amChinh = word[k]
            k = k - 3
            cnt += 1

    elif len(word) - cnt == 1:
        if word[k] in am_chinh:
            amChinh = word[k]
            cnt += 1
            k = k - 1
    else:
        return [phuAmDau, amDem, amChinh, amCuoi]

    print(cnt)
    print (k)
    # Lấy âm đệm
    if len(word) - cnt >= 1: 
        if word[k] in am_dem:
            amDem = word[k]
            k = k - 1
            cnt += 1
    else:
        return [phuAmDau, amDem, amChinh, amCuoi]


    # Lấy âm đầu
    padCount = len(word) - cnt
    i = 0 
    while padCount > 0:
        phu_am_dau += word[i]
        i += 1
        padCount -= 1

    ls = [phuAmDau, amDem, amChinh, amCuoi]
    return ls

In [38]:
phonemeLs = phu_am_dau + am_dem + am_chinh + am_cuoi
phonemeLs = list(set(phonemeLs))
phonemeLs.sort()
print(len(phonemeLs))
print(phonemeLs)

49
['a', 'b', 'c', 'ch', 'd', 'e', 'g', 'gh', 'gi', 'h', 'i', 'ia', 'iê', 'k', 'kh', 'l', 'm', 'n', 'ng', 'ngh', 'nh', 'o', 'oo', 'p', 'ph', 'q', 'r', 's', 't', 'th', 'tr', 'u', 'ua', 'uê', 'uô', 'v', 'x', 'y', 'ya', 'yê', 'â', 'ê', 'ô', 'ă', 'đ', 'ơ', 'ư', 'ưa', 'ươ']


In [39]:
# Create phoneme dataframe
lst = [[0]*len(phonemeLs)]*len(phonemeLs)
df = pd.DataFrame(lst, columns=phonemeLs, index=phonemeLs)
df

,a,b,c,ch,d,e,g,gh,gi,h,...,yê,â,ê,ô,ă,đ,ơ,ư,ưa,ươ
a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ch,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
h,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# print(len(wordlist))
# print(wordlist)

In [12]:
for word in wordlist:
    if (len(word) > 1):
        print(word)
        word = separatePhoneme(word)
        for i in range (len(word)-1):
            df[word[i+1]][word[i]] += 1 
df

di
di
di


KeyError: -1

In [213]:
for word in wordlist:
    if (len(word) > 1):
        for i in range(len(word)-1):
            df[word[i+1]][word[i]] += 1
df.head(5)

,a,b,c,d,e,f,g,h,i,j,...,x,y,z,â,ê,ô,ă,đ,ơ,ư
a,3,14,1303,15,2,6,13,5,2327,1,...,15,923,10,0,0,0,0,10,0,0
b,1480,0,2,1,160,0,0,0,779,0,...,0,1,0,313,203,494,227,0,110,146
c,1409,10,8,3,18,0,1,4890,19,1,...,4,3,1,569,0,760,160,2,234,531
d,563,0,0,0,77,0,0,0,426,0,...,0,3,0,469,62,76,69,0,67,286
e,3,3,44,5,3,1,2,4,10,0,...,4,0,4,0,0,0,0,1,0,0


### Bình diện ngữ âm: Những cặp âm vị thường đi chung với nhau:

In [216]:
PhonemePairs = df.idxmax(axis=1)
numOfRepeat = df.max(axis=1)
data = {"Phoneme Pairs": PhonemePairs,
        "number Of repetitions:": numOfRepeat}
data = pd.concat(data, axis = 1)
data.sort_values(by='number Of repetitions:', ascending=False)

,Phoneme Pairs,number Of repetitions:
n,g,11719
a,n,5470
c,h,4890
t,h,4501
i,ê,4021
h,i,3433
ê,n,3179
ô,n,2922
ư,ơ,2616
â,n,2452


In [218]:
from dataclasses import replace

f = open('VDic_uni.txt', encoding="utf8")
wordTypeLs = []
while True:
    line = f.readline()
    if not line:
        break 
    tmp = line.split("\t")[-1].replace("\n", "").split(", ")
    wordTypeLs += tmp
    
wordTypeLs = [s.replace(" ", "") for s in wordTypeLs if len(s) != 0 and s[0] not in invalid_char and len(s) < 6]
wordTypeLs = list(set(wordTypeLs))
wordTypeLs.sort()

print(len(wordTypeLs))
print(wordTypeLs)    

49
['Aa', 'An', 'Cc', 'Cm', 'E', 'I', 'Ja', 'Jd', 'Ji', 'Jr', 'Jt', 'Jt,Jd', 'Na', 'Nc', 'NcVt', 'Ng', 'Nl', 'Nn', 'Np', 'Nt', 'Nu', 'Pa', 'Pd', 'Pi', 'Pn', 'Pp', 'Va', 'Vb', 'Vc', 'Vim', 'Vit', 'Vitb', 'Vitc', 'Vitim', 'Vits', 'Vla', 'Vo', 'Vs', 'Vt', 'Vta', 'Vtb', 'Vtc', 'Vtim', 'Vto', 'Vts', 'Vtv', 'Vv', 'X', 'đg']


In [220]:
lst = [[0]*len(wordTypeLs)]*len(phonemeLs)
wordType_df = pd.DataFrame(lst, columns=wordTypeLs, index=phonemeLs)
wordType_df

,Aa,An,Cc,Cm,E,I,Ja,Jd,Ji,Jr,...,Vta,Vtb,Vtc,Vtim,Vto,Vts,Vtv,Vv,X,đg
a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
h,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
j,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


https://vnlp.net/ti%E1%BA%BFng-vi%E1%BB%87t-c%C6%A1-b%E1%BA%A3n/am-t%E1%BB%91-am-v%E1%BB%8B-ti%E1%BA%BFng-hinh-v%E1%BB%8B/

https://vnlp.net/ti%e1%ba%bfng-vi%e1%bb%87t-c%c6%a1-b%e1%ba%a3n/h%e1%bb%87-th%e1%bb%91ng-am-v%e1%bb%8b/

